In [1]:
import matplotlib.pyplot as plt # type: ignore
import numpy as np # type: ignore
import os
import pandas as pd # type: ignore
import requests
from bs4 import BeautifulSoup
import pandas as pd

# prep Titan Builder public key from their page
url = 'https://docs.titanbuilder.xyz/builder-public-keys'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

assorted_list = [tag.text for tag in soup.find_all('li')]
titan_pub_key_list = [key for key in assorted_list if key.startswith('0x')]


In [2]:
# Read 921 blocks' bidding data. Titan won these 921 blocks.
pd.options.display.float_format = '{:.0f}'.format
# Read the titan 921 block bidding history parquet file
dft = pd.read_parquet('titan921.parquet', engine='pyarrow')

In [3]:
# Load winning bid block history data
#  !!!the data here is NOT THE LATEST FROM THE OG MEV DATA ALWAYS PROJECT. Go to: https://github.com/dataalways/mevboost-data to sync.
base_path = '../data/'
file_paths = os.listdir(base_path)

dfs = []
for file in file_paths:
    if len(file) < 10: #.DS_store
        continue
    
    df_tmp = pd.read_parquet(os.path.join(base_path, file))
    dfs.append(df_tmp)

df = pd.concat(dfs)
df = df[df['payload_delivered'] == True]
df.sort_values(by=['block_number', 'bid_timestamp_ms'], ascending=True, inplace=True)
df.reset_index(inplace=True, drop=True)